# Limpieza y Análisis de Datos - Semana 3

In [20]:
# Importamos las librerias necesarias 
import pandas as pd
import matplotlib.pyplot as plt

## Lectura de datos

## Limpieza de Datos

## --- Viajes ---
#### Preguntas h, j, t

## --- Ciudades ---
#### Preguntas o-s, f

## --- Tiendas ---
#### Preguntas l - s

## --- Vehiculos --- 
#### Preguntas a - e, g, i, k